<a href="https://colab.research.google.com/github/FunmilolaAdetuwo/Final-Project/blob/main/Analyzing_the_Impact_of_Mergers_and_Acquisitions_on_Stock_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Importing necessary libraries

import yfinance as yf
import pandas as pd
from datetime import datetime
import os
import re


Let's list the name of the companies for the different Merger and Acquisition(M&A) types

For the stock price years we will be using 5 year stock price, which will be 2.5 years pre M&A date and 2.5 years post M&A dates
the companies that will be used are: Microsoft which acquired LinkedIn (for Horizontal type of M&A)
Amazon which acquired Whole Foods (for Vertical type of M&A)
Berkshire Hathaway which acquired Precision Castparts (for Conglomerate type of M&A)


In [3]:
# First, let's diplay the company tickers,acquisition announcement dates, and the SEC filing links from SEC EDGAR
acqsitn_dtls = {
    "Microsoft Corporation (MSFT)": {
        "Date": "June 13, 2016",
        "SEC Filing": "https://www.sec.gov/Archives/edgar/data/789019/000119312516619689/d116815d8k.htm"
    },
    "Amazon.com, Inc. (AMZN)": {
        "Date": "June 16, 2017",
        "SEC Filing": "https://www.sec.gov/Archives/edgar/data/1018724/000119312517205287/d352949d8k.htm"
    },
    "Berkshire Hathaway Inc. (BRK-A)": {
        "Date": "August 10, 2015",
        "SEC Filing": "https://www.sec.gov/Archives/edgar/data/1067983/000119312515285104/d41723d8k.htm"
    }
}

print("M&A announcement dates:")
for company, details in acqsitn_dtls.items():
    print(f"{company}: {details['Date']}")
    print(f"SEC Filing Link: {details['SEC Filing']}\n")


M&A announcement dates:
Microsoft Corporation (MSFT): June 13, 2016
SEC Filing Link: https://www.sec.gov/Archives/edgar/data/789019/000119312516619689/d116815d8k.htm

Amazon.com, Inc. (AMZN): June 16, 2017
SEC Filing Link: https://www.sec.gov/Archives/edgar/data/1018724/000119312517205287/d352949d8k.htm

Berkshire Hathaway Inc. (BRK-A): August 10, 2015
SEC Filing Link: https://www.sec.gov/Archives/edgar/data/1067983/000119312515285104/d41723d8k.htm



In [6]:
# Showing the stock price period to be used for the different mergerand acquisition types
cmpys = {
    'horizontal': {'company': 'MSFT', 'ma_dte': '2016-06-13', 'start_date': '2013-12-13', 'end_date': '2018-12-13'},
    'vertical': {'company': 'AMZN', 'ma_dte': '2017-06-16', 'start_date': '2014-12-16', 'end_date': '2019-12-16'},
    'conglomerate': {'company': 'BRK-A', 'ma_dte': '2015-08-10', 'start_date': '2013-02-10', 'end_date': '2018-02-10'}
}



In [11]:
# Downloading the stock price data from Yahoo Finance for 2.5 years before and 2.5 years after M&A
def download_stock_data(ticker, start_date, end_date):
    stk_dt = yf.download(ticker, start=start_date, end=end_date)
    return stk_dt

# Collecting the stock data for each of the M&A type
stk_dt = {}
for mna_type, company_info in cmpys.items():
    ticker = company_info['company']
    start_date = company_info['start_date']
    end_date = company_info['end_date']
    stk_dt[mna_type] = download_stock_data(ticker, start_date, end_date)

mna_dates = pd.DataFrame(cmpys).T
mna_dates['ma_dte'] = pd.to_datetime(mna_dates['ma_dte'])

# Displaying the stock price data
print(stk_dt)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

{'horizontal': Price                       Adj Close       Close        High         Low  \
Ticker                           MSFT        MSFT        MSFT        MSFT   
Date                                                                        
2013-12-13 00:00:00+00:00   30.727118   36.689999   37.450001   36.619999   
2013-12-16 00:00:00+00:00   30.894609   36.889999   37.000000   36.540001   
2013-12-17 00:00:00+00:00   30.584745   36.520000   37.110001   36.330002   
2013-12-18 00:00:00+00:00   30.635002   36.580002   36.599998   35.529999   
2013-12-19 00:00:00+00:00   30.358637   36.250000   36.549999   36.080002   
...                               ...         ...         ...         ...   
2018-12-06 00:00:00+00:00  103.122932  109.190002  109.239998  105.000000   
2018-12-07 00:00:00+00:00   98.995735  104.820000  109.449997  104.300003   
2018-12-10 00:00:00+00:00  101.611816  107.589996  107.980003  103.889999   
2018-12-11 00:00:00+00:00  102.556244  108.589996  110.949997